# Final Project: SQL

#### Connecting to the Database:

In [1]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine


db_config = {'user': 'practicum_student',         # username
             'pwd': 's65BlTKV3faNIGhmvJVzOqhs', # password
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432,              # connection port
             'db': 'data-analyst-final-project-db'}          # the name of the database

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                                     db_config['pwd'],
                                                                       db_config['host'],
                                                                       db_config['port'],
                                                                       db_config['db'])

engine = create_engine(connection_string, connect_args={'sslmode':'require'})

### Context:

The coronavirus took the entire world by surprise, changing everyone's daily routine. City dwellers no longer spent their free time outside, going to cafes and malls; more people were at home, reading books. That attracted the attention of startups that rushed to develop new apps for book lovers. 

You've been given a database of one of the services competing in this market. It contains data on books, publishers, authors, and customer ratings and reviews of books. This information will be used to generate a value proposition for a new product.

### Main goal of this study: 

As the context indicates, many people started reading books again during the pandemic. In order to establish reading habits even after the pandemic, people need books that make them want to keep reading and not return to cinemas and cafés or watch (even more) Netflix. 

These tables provide a good overview of which books readers prefer and which characteristics tend to keep people from reading more because they simply don't like the books. The better the books are, the more likely they are to read more. 

Below I will try to suggest a book that best fits the reader's preferences in order to develop the best-possible app.

### Displaying the first rows:

In [11]:
pd.io.sql.read_sql('SELECT * FROM books LIMIT 5', con = engine)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [3]:
pd.io.sql.read_sql('SELECT * FROM authors LIMIT 5', con = engine)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [4]:
pd.io.sql.read_sql('SELECT * FROM ratings LIMIT 5', con = engine)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [5]:
pd.io.sql.read_sql('SELECT * FROM reviews LIMIT 5', con = engine)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


In [6]:
pd.io.sql.read_sql('SELECT * FROM publishers LIMIT 5', con = engine)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


### Defining a function to run the queries.

In [57]:
def sql(query):
    return pd.io.sql.read_sql(query, con = engine)

### Q1: Find the number of books released after January 1, 2000.

In [86]:
sql("SELECT COUNT(book_id) AS number_books_after_1999 FROM books WHERE publication_date >= '2000-01-01' ")

,number_books_after_1999
0,821


The publication date ranges from 1952 to 2020. 821 books which were published after 1999, show that many books are still published and it is very important to filter out the good ones.

### Q2: Find the number of user reviews and the average rating for each book.

In [97]:
sql('SELECT COUNT(reviews.review_id) AS number_of_reviews, ROUND(AVG(ratings.rating),2) AS average_rating_score FROM ratings INNER JOIN reviews ON ratings.book_id = reviews.book_id')

,number_of_reviews,average_rating_score
0,26167,3.94


Full code: sql('SELECT COUNT(reviews.review_id) AS number_of_reviews, ROUND(AVG(ratings.rating),2) AS average_rating_score FROM ratings INNER JOIN reviews ON ratings.book_id = reviews.book_id')

Since there are 1000 books, the number of reviews shows that there are more than 26 reviews per book on average, so definitely
enough to analyze them carefully. This could be done using Natural Language Processing in Python, for example.
It also shows that all users are quite active and engaged. An app developer for books should try to target the engaged users, as higher traffic will increase the popularity of the app.

The average rating number ranges from 1 (lowest possible number) to 5 (highest possible number). An average of almost 4 shows that the books were generally rated quite positively. However, this also means that if you are looking for really good books, they should have an average rating close to 5, because 4 is "only" average.

### Q3: Identify the publisher that has released the greatest number of books with more than 50 pages (this will help you exclude brochures and similar publications from your analysis).

In [77]:
sql('SELECT COUNT(books.book_id) AS number_books, publishers.publisher FROM books INNER JOIN publishers USING (publisher_id) WHERE books.num_pages > 50 GROUP BY publishers.publisher ORDER BY COUNT(books.book_id) DESC LIMIT 1')

,number_books,publisher
0,42,Penguin Books


Full code: sql('SELECT COUNT(books.book_id) AS number_books, publishers.publisher FROM books INNER JOIN publishers USING (publisher_id) WHERE books.num_pages > 50 GROUP BY publishers.publisher ORDER BY COUNT(books.book_id) DESC LIMIT 1')

A total number of 42 books published by the publisher "Penguin Books" sounds impressive at first glance, but it is very important to evaluate not only quantity but also quality. The query below shows that the average rating of the books published by "Penguin Books" is below average (see Task 2) and thus less attractive for our app than one might first assume.

In [103]:
sql("SELECT ROUND(AVG(rating),2) as avg_rating_penguin_books FROM ratings JOIN books ON ratings.book_id=books.book_id JOIN publishers ON books.publisher_id = publishers.publisher_id WHERE publisher='Penguin Books'")

,avg_rating_penguin_books
0,3.7


Full code: sql("SELECT ROUND(AVG(rating),2) as avg_rating_penguin_books FROM ratings JOIN books ON ratings.book_id=books.book_id JOIN publishers ON books.publisher_id = publishers.publisher_id WHERE publisher='Penguin Books'")

### Q4: Identify the author with the highest average book rating (look only at books with at least 50 ratings).

In [78]:
sql('SELECT authors.author, ROUND(AVG(ratings.rating),2) AS average_rating FROM authors INNER JOIN books ON authors.author_id = books.author_id INNER JOIN ratings ON books.book_id = ratings.book_id GROUP BY authors.author HAVING COUNT(ratings.rating_id) > 50 ORDER BY ROUND(AVG(ratings.rating),2) DESC LIMIT 1')

,author,average_rating
0,J.K. Rowling/Mary GrandPré,4.29


Full code: sql('SELECT authors.author, ROUND(AVG(ratings.rating),2) AS average_rating FROM authors INNER JOIN books ON authors.author_id = books.author_id INNER JOIN ratings ON books.book_id = ratings.book_id GROUP BY authors.author HAVING COUNT(ratings.rating_id) > 50 ORDER BY ROUND(AVG(ratings.rating),2) DESC LIMIT 1')

This key figure is very important for our application. Our application should incorporate a large range of books from J.K. Rowling/Mary GrandPré because customers tend to spend more time with the application if they read books they actually like.

### Q5: Find the average number of text reviews among users who rated more than 50 books. 

In [79]:
sql('with t1 as (SELECT reviews.username, COUNT(reviews.text) AS number_reviews FROM reviews INNER JOIN ratings ON reviews.book_id = ratings.book_id GROUP BY reviews.username HAVING COUNT(ratings.rating_id) > 50) SELECT ROUND(AVG(number_reviews),2) AS average_reviews FROM t1')

,average_reviews
0,166.68


Full code: sql('with t1 as (SELECT reviews.username, COUNT(reviews.text) AS number_reviews FROM reviews INNER JOIN ratings ON reviews.book_id = ratings.book_id GROUP BY reviews.username HAVING COUNT(ratings.rating_id) > 50) SELECT ROUND(AVG(number_reviews),2) AS average_reviews FROM t1')

This insight more or less corresponds to the insights of the second task. Our application must encourage users to engage in order to increase traffic on our platform.

### Summary:

The various questions made it clear which aspects are important when launching an application that includes books.

The application needs a wide range of books, well-known and well-rated authors, and renowned publishers. But no less important are interactive users and reviewers who can frequently rate books, engage and also recommend our application.